# To save uncertainty results for downstream task

In [1]:
RUN_ID = 32
INPUT_MODALITY = "T1_T2_FLAIR"
RANDOM_SEED = 0
ROOT_DIR = "/scratch1/sachinsa/cont_syn"

In [2]:
import os
import pandas as pd

import pdb
import numpy as np
from utils.logger import Logger

import torch
from torch.utils.data import DataLoader

from utils.model import create_UNet3D, inference
from utils.transforms import contr_syn_transform_2 as data_transform
from utils.dataset import BraTSDataset

logger = Logger(log_level='DEBUG')

[INFO] Logging level is: DEBUG


In [3]:
load_dir = os.path.join(ROOT_DIR, f"run_{RUN_ID}")
save_dir = os.path.join('/scratch1/sachinsa/data/contr_generated', f"run_{RUN_ID}")
os.makedirs(save_dir, exist_ok=True)

In [4]:
input_filter = []
if INPUT_MODALITY == "ONLY_T1":
    input_filter = [1]
elif INPUT_MODALITY == "T1_T2":
    input_filter = [1,3]
elif INPUT_MODALITY == "T1_T2_FLAIR":
    input_filter = [0,1,3]

In [5]:
device = torch.device("cuda:0")
out_channels = 1
model = create_UNet3D(len(input_filter), out_channels, device)

In [6]:
all_dataset = BraTSDataset(
    version='2017',
    section='all',
    seed = RANDOM_SEED,
    transform = data_transform['val']
)
all_loader = DataLoader(all_dataset, batch_size=1, shuffle=False, num_workers=8)

In [7]:
# # Load masks
# mask_root_dir = "/scratch1/sachinsa/data/masks/brats2017"
# train_mask_df = pd.read_csv(os.path.join(mask_root_dir, "train_mask.csv"), index_col=0)
# val_mask_df = pd.read_csv(os.path.join(mask_root_dir, "val_mask.csv"), index_col=0)
# all_mask_df = pd.concat([train_mask_df, val_mask_df], axis=0)
# all_mask_df.head(2)

In [8]:
checkpoint = torch.load(os.path.join(load_dir, 'best_checkpoint.pth'), weights_only=True)
model.load_state_dict(checkpoint['model_state_dict'])
model.eval()
channels = ["FLAIR", "T1w", "T1Gd", "T2w"]

In [9]:
import nibabel as nib

In [ ]:
with torch.no_grad():
    for this_data in all_loader:
        this_inputs, this_ids = (
            this_data["image"].to(device),
            this_data["id"],
        )
        this_inputs = this_inputs[:,input_filter, ...]
        this_outputs = inference(this_inputs, model)
        
        mri_array = this_outputs[0].detach().permute(1, 2, 3, 0).cpu().numpy()
        nifti_img = nib.Nifti1Image(mri_array,affine=np.eye(4))
        output_filename = os.path.join(save_dir, f'BRATS_{this_ids[0]}.nii.gz')
        print(output_filename)
        nib.save(nifti_img, output_filename)

/scratch1/sachinsa/data/contr_generated/run_32/BRATS_457.nii.gz
/scratch1/sachinsa/data/contr_generated/run_32/BRATS_306.nii.gz
